In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder


In [2]:
#mount google drive

from google.colab import drive

drive.mount('/content/drive')

path = "/content/drive/MyDrive/"


Mounted at /content/drive


In [3]:
combinedDf = pd.read_csv(path + 'updated_Combined.csv')

combinedDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215890 entries, 0 to 1215889
Data columns (total 24 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Unnamed: 0   1215890 non-null  float64
 1   Seq          1215890 non-null  float64
 2   Dur          1215890 non-null  float64
 3   RunTime      1215890 non-null  float64
 4   Mean         1215890 non-null  float64
 5   Sum          1215890 non-null  float64
 6   Min          1215890 non-null  float64
 7   Max          1215890 non-null  float64
 8   TotPkts      1215890 non-null  float64
 9   SrcPkts      1215890 non-null  float64
 10  DstPkts      1215890 non-null  float64
 11  TotBytes     1215890 non-null  float64
 12  SrcBytes     1215890 non-null  float64
 13  DstBytes     1215890 non-null  float64
 14  Offset       1215890 non-null  float64
 15  sMeanPktSz   1215890 non-null  float64
 16  dMeanPktSz   1215890 non-null  float64
 17  Load         1215890 non-null  float64
 18  Sr

In [4]:
features = ['Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max',
            'TotPkts', 'SrcPkts', 'DstPkts', 'TotBytes', 'SrcBytes',
            'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz',
            'Load', 'SrcLoad', 'Rate', 'SrcRate']
target = 'Label'

In [5]:
# Load the trained models from files

models_path = path+'models/'

dt_model = joblib.load(models_path + 'dt_model.pkl')
logistic_model = joblib.load(models_path + 'logistic_model.pkl')
mlp_model = joblib.load(models_path + 'mlp_model.pkl')
rf_model = joblib.load(models_path + 'rf_model.pkl')
knn_model = joblib.load(models_path + 'knn_model.pkl')
gbm_model = joblib.load(models_path + 'gbm.pkl')
